In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

train_dir="../data/train/"
test_dir="../data/test/"

link_dir="../data/"

%pwd

'/home/cseadmin/undergrad1/dz/GISCUP2021/data_process'

In [3]:
def gen_feats(task, head, path, nl, link_attr, link_time_attr, driver_attr, slice_attr, link_status_attr, OD_attr):
########################
###head features
########################
    head['speed'] = head['distance']/head['simple_eta']
    head['speed_avg'] = head['slice_id'].map(head.groupby('slice_id')['speed'].mean())
    head['speed_std'] = head['slice_id'].map(head.groupby('slice_id')['speed'].std())
    head['speed_min'] = head['slice_id'].map(head.groupby('slice_id')['speed'].min())
    head['speed_max'] = head['slice_id'].map(head.groupby('slice_id')['speed'].max())
    head['volume'] = head['slice_id'].map(head.groupby('slice_id').size())
    head['old_driver'] = head['driver_id'].map(head.groupby('driver_id').size())

# -----------------------------------------------------------------------------
    head['driver_count'] = head['driver_id'].map(driver_attr['count'])

    head['driver_ata_minus_simple_eta_std'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_std'])
    head['driver_ata_minus_simple_eta_avg'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_avg'])
    head['driver_ata_minus_simple_eta_min'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_min'])
    head['driver_ata_minus_simple_eta_max'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_max'])
    head['driver_ata_minus_simple_eta_skew'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_skew'])
    head['driver_ata_minus_simple_eta_kurt'] = head['driver_id'].map(driver_attr['ata_minus_simple_eta_kurt'])

    # head['driver_ata_std'] = head['driver_id'].map(driver_attr['ata_std'])
    # head['driver_ata_avg'] = head['driver_id'].map(driver_attr['ata_avg'])
    # head['driver_ata_min'] = head['driver_id'].map(driver_attr['ata_min'])
    # head['driver_ata_max'] = head['driver_id'].map(driver_attr['ata_max'])
    # head['driver_ata_skew'] = head['driver_id'].map(driver_attr['ata_skew'])
    # head['driver_ata_kurt'] = head['driver_id'].map(driver_attr['ata_kurt'])

    # head['driver_distance_std'] = head['driver_id'].map(driver_attr['distance_std'])
    # head['driver_distance_avg'] = head['driver_id'].map(driver_attr['distance_avg'])
    # head['driver_distance_min'] = head['driver_id'].map(driver_attr['distance_min'])
    # head['driver_distance_max'] = head['driver_id'].map(driver_attr['distance_max'])
    # head['driver_distance_skew'] = head['driver_id'].map(driver_attr['distance_skew'])
    # head['driver_distance_kurt'] = head['driver_id'].map(driver_attr['distance_kurt'])

    # head['driver_simple_eta_std'] = head['driver_id'].map(driver_attr['simple_eta_std'])
    # head['driver_simple_eta_avg'] = head['driver_id'].map(driver_attr['simple_eta_avg'])
    # head['driver_simple_eta_min'] = head['driver_id'].map(driver_attr['simple_eta_min'])
    # head['driver_simple_eta_max'] = head['driver_id'].map(driver_attr['simple_eta_max'])
    # head['driver_simple_eta_skew'] = head['driver_id'].map(driver_attr['simple_eta_skew'])
    # head['driver_simple_eta_kurt'] = head['driver_id'].map(driver_attr['simple_eta_kurt'])

    # head['driver_slice_id_std'] = head['driver_id'].map(driver_attr['slice_id_std'])
    # head['driver_slice_id_avg'] = head['driver_id'].map(driver_attr['slice_id_avg'])
    # head['driver_slice_id_min'] = head['driver_id'].map(driver_attr['slice_id_min'])
    # head['driver_slice_id_max'] = head['driver_id'].map(driver_attr['slice_id_max'])
    # head['driver_slice_id_skew'] = head['driver_id'].map(driver_attr['slice_id_skew'])
    # head['driver_slice_id_kurt'] = head['driver_id'].map(driver_attr['slice_id_kurt'])

    # for c in driver_attr.columns:
    #     head['driver_'+c] = head['driver_id'].map(driver_attr[c])
# -----------------------------------------------------------------------------

    head['slice_avg'] = head['slice_id'].map(slice_attr['avg'])
    head['slice_std'] = head['slice_id'].map(slice_attr['std'])
    head['slice_min'] = head['slice_id'].map(slice_attr['min'])
    head['slice_max'] = head['slice_id'].map(slice_attr['max'])
    head['slice_count'] = head['slice_id'].map(slice_attr['count'])
    head['slice_skew'] = head['slice_id'].map(slice_attr['skew'])
    head['slice_kurt'] = head['slice_id'].map(slice_attr['kurt'])

    head.set_index('order_id', inplace=True)

    path['link_eta'] = path['link_time']*path['link_ratio']
    path['cum_time'] = (path.groupby('order_id')['link_eta'].cumsum()/300).astype('int')
    link = path[path['cross_flag']==0]
    cross = path[path['cross_flag']==1]
########################
###link features
########################
    link = link[link['link_time']>0]
    link['link_id'] = link['link_id'].astype('int')
    link['link_type'] = link['link_id'].map(link_attr['link_label']).fillna(0).astype('int')
    link['next_link'] = link['link_id'].map(nl).fillna(0).astype('int')
    link['slice_id'] = link['order_id'].map(head['slice_id'])

    link["link_time_std"] = link["link_id"].map(link_time_attr["std"])
    link["link_time_avg"] = link["link_id"].map(link_time_attr["avg"])
    link["link_time_min"] = link["link_id"].map(link_time_attr["min"])
    link["link_time_max"] = link["link_id"].map(link_time_attr["max"])
    link["link_time_count"] = link["link_id"].map(link_time_attr["count"])
    link["link_time_skew"] = link["link_id"].map(link_time_attr["skew"])
    link["link_time_kurt"] = link["link_id"].map(link_time_attr["kurt"])
    link['link_time_type'] = link['link_id'].map(link_time_attr['label']).fillna(0).astype('int')

    link["link_cur_status_0"] = link["link_id"].map(link_status_attr["current_status_0"])
    link["link_cur_status_1"] = link["link_id"].map(link_status_attr["current_status_1"])
    link["link_cur_status_2"] = link["link_id"].map(link_status_attr["current_status_2"])
    link["link_cur_status_3"] = link["link_id"].map(link_status_attr["current_status_3"])
    link["link_cur_status_4"] = link["link_id"].map(link_status_attr["current_status_4"])

    link["link_arr_status_0"] = link["link_id"].map(link_status_attr["arrival_status_0"])
    link["link_arr_status_1"] = link["link_id"].map(link_status_attr["arrival_status_1"])
    link["link_arr_status_2"] = link["link_id"].map(link_status_attr["arrival_status_2"])
    link["link_arr_status_3"] = link["link_id"].map(link_status_attr["arrival_status_3"])
    link["link_arr_status_4"] = link["link_id"].map(link_status_attr["arrival_status_4"])

    link["link_cur_flag"] = link["link_id"].map(link_status_attr["cur_flag"])
    link["link_arr_flag"] = link["link_id"].map(link_status_attr["arr_flag"])

    gl = link.groupby('order_id')
    head['link_no'] = head.index.map(gl.size())
    head['link_time_sum'] = head.index.map(gl['link_eta'].sum())
    head['link_time_avg'] = head.index.map(gl['link_time'].mean())
    head['link_time_std'] = head.index.map(gl['link_time'].std())
    head['link_time_max'] = head.index.map(gl['link_time'].max())
    head['link_time_min'] = head.index.map(gl['link_time'].min())

    head['o_count'] = head.index.map(gl['link_id'].first().map(OD_attr["o_count"]))
    head['d_count'] = head.index.map(gl['link_id'].last().map(OD_attr["d_count"]))

# -----------------------------------------------------------------------------
    head['link_time_std_sum'] = head.index.map(gl['link_time_std'].sum())
    head['link_time_std_avg'] = head.index.map(gl['link_time_std'].mean())
    head['link_time_std_std'] = head.index.map(gl['link_time_std'].std())
    head['link_time_std_min'] = head.index.map(gl['link_time_std'].min())
    head['link_time_std_max'] = head.index.map(gl['link_time_std'].max())

    head['link_time_avg_sum'] = head.index.map(gl['link_time_avg'].sum())
    head['link_time_avg_avg'] = head.index.map(gl['link_time_avg'].mean())
    head['link_time_avg_std'] = head.index.map(gl['link_time_avg'].std())
    head['link_time_avg_min'] = head.index.map(gl['link_time_avg'].min())
    head['link_time_avg_max'] = head.index.map(gl['link_time_avg'].max())

    head['link_time_max_sum'] = head.index.map(gl['link_time_max'].sum())
    head['link_time_max_avg'] = head.index.map(gl['link_time_max'].mean())
    head['link_time_max_std'] = head.index.map(gl['link_time_max'].std())
    head['link_time_max_min'] = head.index.map(gl['link_time_max'].min())
    head['link_time_max_max'] = head.index.map(gl['link_time_max'].max())

    head['link_time_min_sum'] = head.index.map(gl['link_time_min'].sum())
    head['link_time_min_avg'] = head.index.map(gl['link_time_min'].mean())
    head['link_time_min_std'] = head.index.map(gl['link_time_min'].std())
    head['link_time_min_min'] = head.index.map(gl['link_time_min'].min())
    head['link_time_min_max'] = head.index.map(gl['link_time_min'].max())

    head['link_time_count_sum'] = head.index.map(gl['link_time_count'].sum())
    head['link_time_count_avg'] = head.index.map(gl['link_time_count'].mean())
    head['link_time_count_std'] = head.index.map(gl['link_time_count'].std())
    head['link_time_count_min'] = head.index.map(gl['link_time_count'].min())
    head['link_time_count_max'] = head.index.map(gl['link_time_count'].max())

    head['link_time_skew_sum'] = head.index.map(gl['link_time_skew'].sum())
    head['link_time_skew_avg'] = head.index.map(gl['link_time_skew'].mean())
    head['link_time_skew_std'] = head.index.map(gl['link_time_skew'].std())
    head['link_time_skew_min'] = head.index.map(gl['link_time_skew'].min())
    head['link_time_skew_max'] = head.index.map(gl['link_time_skew'].max())

    head['link_time_kurt_sum'] = head.index.map(gl['link_time_kurt'].sum())
    head['link_time_kurt_avg'] = head.index.map(gl['link_time_kurt'].mean())
    head['link_time_kurt_std'] = head.index.map(gl['link_time_kurt'].std())
    head['link_time_kurt_min'] = head.index.map(gl['link_time_kurt'].min())
    head['link_time_kurt_max'] = head.index.map(gl['link_time_kurt'].max())
# -----------------------------------------------------------------------------

    head['link_cur_status_0_sum'] = head.index.map(gl['link_cur_status_0'].sum())
    head['link_cur_status_0_avg'] = head.index.map(gl['link_cur_status_0'].mean())
    head['link_cur_status_0_std'] = head.index.map(gl['link_cur_status_0'].std())
    head['link_cur_status_0_min'] = head.index.map(gl['link_cur_status_0'].min())
    head['link_cur_status_0_max'] = head.index.map(gl['link_cur_status_0'].max())

    head['link_cur_status_1_sum'] = head.index.map(gl['link_cur_status_1'].sum())
    head['link_cur_status_1_avg'] = head.index.map(gl['link_cur_status_1'].mean())
    head['link_cur_status_1_std'] = head.index.map(gl['link_cur_status_1'].std())
    head['link_cur_status_1_min'] = head.index.map(gl['link_cur_status_1'].min())
    head['link_cur_status_1_max'] = head.index.map(gl['link_cur_status_1'].max())

    head['link_cur_status_2_sum'] = head.index.map(gl['link_cur_status_2'].sum())
    head['link_cur_status_2_avg'] = head.index.map(gl['link_cur_status_2'].mean())
    head['link_cur_status_2_std'] = head.index.map(gl['link_cur_status_2'].std())
    head['link_cur_status_2_min'] = head.index.map(gl['link_cur_status_2'].min())
    head['link_cur_status_2_max'] = head.index.map(gl['link_cur_status_2'].max())

    head['link_cur_status_3_sum'] = head.index.map(gl['link_cur_status_3'].sum())
    head['link_cur_status_3_avg'] = head.index.map(gl['link_cur_status_3'].mean())
    head['link_cur_status_3_std'] = head.index.map(gl['link_cur_status_3'].std())
    head['link_cur_status_3_min'] = head.index.map(gl['link_cur_status_3'].min())
    head['link_cur_status_3_max'] = head.index.map(gl['link_cur_status_3'].max())

    head['link_cur_status_4_sum'] = head.index.map(gl['link_cur_status_4'].sum())
    head['link_cur_status_4_avg'] = head.index.map(gl['link_cur_status_4'].mean())
    head['link_cur_status_4_std'] = head.index.map(gl['link_cur_status_4'].std())
    head['link_cur_status_4_min'] = head.index.map(gl['link_cur_status_4'].min())
    head['link_cur_status_4_max'] = head.index.map(gl['link_cur_status_4'].max())

    head['link_arr_status_0_sum'] = head.index.map(gl['link_arr_status_0'].sum())
    head['link_arr_status_0_avg'] = head.index.map(gl['link_arr_status_0'].mean())
    head['link_arr_status_0_std'] = head.index.map(gl['link_arr_status_0'].std())
    head['link_arr_status_0_min'] = head.index.map(gl['link_arr_status_0'].min())
    head['link_arr_status_0_max'] = head.index.map(gl['link_arr_status_0'].max())

    head['link_arr_status_1_sum'] = head.index.map(gl['link_arr_status_1'].sum())
    head['link_arr_status_1_avg'] = head.index.map(gl['link_arr_status_1'].mean())
    head['link_arr_status_1_std'] = head.index.map(gl['link_arr_status_1'].std())
    head['link_arr_status_1_min'] = head.index.map(gl['link_arr_status_1'].min())
    head['link_arr_status_1_max'] = head.index.map(gl['link_arr_status_1'].max())

    head['link_arr_status_2_sum'] = head.index.map(gl['link_arr_status_2'].sum())
    head['link_arr_status_2_avg'] = head.index.map(gl['link_arr_status_2'].mean())
    head['link_arr_status_2_std'] = head.index.map(gl['link_arr_status_2'].std())
    head['link_arr_status_2_min'] = head.index.map(gl['link_arr_status_2'].min())
    head['link_arr_status_2_max'] = head.index.map(gl['link_arr_status_2'].max())

    head['link_arr_status_3_sum'] = head.index.map(gl['link_arr_status_3'].sum())
    head['link_arr_status_3_avg'] = head.index.map(gl['link_arr_status_3'].mean())
    head['link_arr_status_3_std'] = head.index.map(gl['link_arr_status_3'].std())
    head['link_arr_status_3_min'] = head.index.map(gl['link_arr_status_3'].min())
    head['link_arr_status_3_max'] = head.index.map(gl['link_arr_status_3'].max())

    head['link_arr_status_4_sum'] = head.index.map(gl['link_arr_status_4'].sum())
    head['link_arr_status_4_avg'] = head.index.map(gl['link_arr_status_4'].mean())
    head['link_arr_status_4_std'] = head.index.map(gl['link_arr_status_4'].std())
    head['link_arr_status_4_min'] = head.index.map(gl['link_arr_status_4'].min())
    head['link_arr_status_4_max'] = head.index.map(gl['link_arr_status_4'].max())

    head['link_cur_flag_sum'] = head.index.map(gl['link_cur_flag'].sum())
    head['link_cur_flag_avg'] = head.index.map(gl['link_cur_flag'].mean())

    head['link_arr_flag_sum'] = head.index.map(gl['link_arr_flag'].sum())
    head['link_arr_flag_avg'] = head.index.map(gl['link_arr_flag'].mean())
# -----------------------------------------------------------------------------

    head['time_delay_max'] = head.index.map(gl['cum_time'].max())
    head['time_delay_avg'] = head.index.map(gl['cum_time'].mean())
    head['time_delay_std'] = head.index.map(gl['cum_time'].std())

    gl = link.groupby(['order_id','link_current_status'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('current_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('current_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('current_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('current_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('current_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('current_time_min_'))

    gl = link.groupby(['order_id','next_link'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('next_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('next_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('next_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('next_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('next_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('next_time_min_'))

    gl = link.groupby(['order_id','link_type'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('type_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('type_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('type_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('type_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('type_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('type_time_min_'))

    gl = link.groupby(['order_id','link_time_type'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('time_type_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('time_type_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('time_type_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('time_type_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('time_type_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('time_type_time_min_'))
    
#     gl = link.groupby(['order_id','link_arrival_status'])['link_time']
#     head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('arrival_no_'))
#     head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('arrival_time_sum_'))
#     head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('arrival_time_avg_'))
#     head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('arrival_time_std_'))
#     head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('arrival_time_max_'))
#     head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('arrival_time_min_'))

########################
###cross features
########################
    gc = cross.groupby('order_id') 
    head['cross_no'] = head.index.map(gc.size())
    head['cross_sum'] = head.index.map(gc['link_time'].sum())
    head['cross_avg'] = head.index.map(gc['link_time'].mean())
    head['cross_std'] = head.index.map(gc['link_time'].std())
    head['cross_max'] = head.index.map(gc['link_time'].max())
    head['cross_min'] = head.index.map(gc['link_time'].min())
#     head['cross_ratio'] = head['cross_sum']/head['simple_eta']

    head = head.fillna(0).reset_index().set_index('slice_id')
    gl = link.groupby(['slice_id','link_current_status'])
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('link_status_no_'))
    head = head.fillna(0).reset_index()

    if(task=='train'):
        head['date'] = f[:8]
    return head

In [4]:
network = pd.read_table(link_dir+'nextlinks.txt',sep=' ',names=['link_id','next_link'])
network['next_link_no'] = network['next_link'].apply(lambda z: 4 if len(list(z.split(',')))>4 else len(list(z.split(','))))
nl = network.set_index('link_id')['next_link_no']

In [7]:
link_attr = pd.read_csv(link_dir+'link_attr.csv')
link_attr.set_index('Unnamed: 0',inplace=True)

link_time_attr = pd.read_csv(link_dir+'link_time_attr.csv')
link_time_attr.set_index('Unnamed: 0',inplace=True)

driver_attr = pd.read_csv(link_dir+'driver_attr.csv')
driver_attr.set_index('Unnamed: 0',inplace=True)

slice_attr = pd.read_csv(link_dir+'slice_attr.csv')
slice_attr.set_index('Unnamed: 0',inplace=True)

link_status_attr = pd.read_csv(link_dir+'link_status_attr.csv')
link_status_attr.set_index('Unnamed: 0',inplace=True)

OD_attr = pd.read_csv(link_dir+'OD_attr.csv')
OD_attr.set_index('Unnamed: 0',inplace=True)

driver_attr.head(10)

,ata_minus_simple_eta_std,ata_minus_simple_eta_avg,ata_minus_simple_eta_min,ata_minus_simple_eta_max,ata_minus_simple_eta_skew,ata_minus_simple_eta_kurt,ata_std,ata_avg,ata_min,ata_max,...,simple_eta_max,simple_eta_skew,simple_eta_kurt,slice_id_std,slice_id_avg,slice_id_min,slice_id_max,slice_id_skew,slice_id_kurt,count
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
75457,2303.907258,1257.211789,-240.0,30009.5818,4.305127,29.920131,2303.907258,1257.211789,-240.0,30009.5818,...,30009.5818,4.305127,29.920131,2303.907258,1257.211789,-240.0,30009.5818,4.305127,29.920131,1205.0
19210,2901.116935,1393.908167,-380.0,31449.0652,4.986158,34.197926,2901.116935,1393.908167,-380.0,31449.0652,...,31449.0652,4.986158,34.197926,2901.116935,1393.908167,-380.0,31449.0652,4.986158,34.197926,845.0
38822,3641.572902,1685.287692,-231.0,49976.2813,5.363101,43.717698,3641.572902,1685.287692,-231.0,49976.2813,...,49976.2813,5.363101,43.717698,3641.572902,1685.287692,-231.0,49976.2813,5.363101,43.717698,1020.0
51627,2824.998467,1444.937261,-383.0,22070.1199,3.814225,17.170264,2824.998467,1444.937261,-383.0,22070.1199,...,22070.1199,3.814225,17.170264,2824.998467,1444.937261,-383.0,22070.1199,3.814225,17.170264,480.0
39965,3227.539116,1537.510897,-207.0,40014.7261,5.643093,47.953207,3227.539116,1537.510897,-207.0,40014.7261,...,40014.7261,5.643093,47.953207,3227.539116,1537.510897,-207.0,40014.7261,5.643093,47.953207,590.0
28933,2205.931355,1214.635599,-247.0,27603.9748,4.220348,29.563140,2205.931355,1214.635599,-247.0,27603.9748,...,27603.9748,4.220348,29.563140,2205.931355,1214.635599,-247.0,27603.9748,4.220348,29.563140,1115.0
52023,1877.674236,1069.695779,-435.0,11019.2463,2.661124,7.439636,1877.674236,1069.695779,-435.0,11019.2463,...,11019.2463,2.661124,7.439636,1877.674236,1069.695779,-435.0,11019.2463,2.661124,7.439636,385.0
61650,2762.043230,1475.041785,-241.0,31862.7168,4.325707,27.481921,2762.043230,1475.041785,-241.0,31862.7168,...,31862.7168,4.325707,27.481921,2762.043230,1475.041785,-241.0,31862.7168,4.325707,27.481921,1040.0
42366,3533.842660,1547.610336,-520.0,65906.2382,6.671317,77.324387,3533.842660,1547.610336,-520.0,65906.2382,...,65906.2382,6.671317,77.324387,3533.842660,1547.610336,-520.0,65906.2382,6.671317,77.324387,2060.0


In [6]:
TRAIN_FILES = ['202008'+str(i).zfill(2)+'.csv' for i in range(1,32)]

for i,f in tqdm(enumerate(TRAIN_FILES)):    
    head = pd.read_csv(train_dir+'train_head/'+f)
    path = pd.read_csv(train_dir+'train_path/'+f)

    if(head.shape[0]==0):
        continue
    if i==0:
        train = gen_feats('train', head, path, nl, link_attr, link_time_attr, driver_attr, slice_attr, link_status_attr, OD_attr)
    else:
        train = train.append(gen_feats('train', head, path, nl, link_attr, link_time_attr, driver_attr, slice_attr, link_status_attr, OD_attr))

0it [00:07, ?it/s]


TypeError: sum() got an unexpected keyword argument 'axis'

In [ ]:
head = pd.read_csv(test_dir+'test_head.csv')
path = pd.read_csv(test_dir+'test_path.csv')

test = gen_feats('test', head, path, nl, link_attr, link_time_attr, driver_attr, slice_attr, link_status_attr, OD_attr)
test['date'] = '20200901'

生成并保存 train set 和 test set

In [ ]:
%%time

train['date'] = pd.to_datetime(train['date'].astype('str'))
test['date'] = pd.to_datetime(test['date'])

for df in [train, test]:
    df['day_bias'] = (df['date']-pd.Timestamp(2020,8,1)).dt.days
    df['weekday'] = df['date'].dt.weekday<5
    df['cross_no'] = df['cross_no']*(df['cross_sum']>0)
    df['link_length'] = df['distance']/df['link_no']
    df['log_distance'] = np.log(df['distance'])
    df['log_simple_eta'] = np.log(df['simple_eta'])
    
    for i in range(5):
        df['current_no_ratio_'+str(i)] = df['current_no_'+str(i)]/df['link_no']
        df['current_time_ratio_'+str(i)] = df['current_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
        df['next_no_ratio_'+str(i)] = df['next_no_'+str(i)]/df['link_no']
        df['next_time_ratio_'+str(i)] = df['next_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
    for i in range(4):
        df['type_no_ratio_'+str(i)] = df['type_no_'+str(i)]/df['link_no']
        df['type_time_ratio_'+str(i)] = df['type_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])

CPU times: user 3.34 s, sys: 788 ms, total: 4.12 s
Wall time: 4.12 s


In [ ]:
%%time

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
print(train.info())
print(test.info())

Memory usage after optimization is: 3192.84 MB
Decreased by 73.3%
Memory usage after optimization is: 101.65 MB
Decreased by 74.3%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8651005 entries, 0 to 291609
Columns: 181 entries, slice_id to type_time_ratio_3
dtypes: bool(1), datetime64[ns](1), float16(142), float32(5), int16(22), int32(4), int8(6)
memory usage: 3.1 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288076 entries, 0 to 288075
Columns: 181 entries, slice_id to type_time_ratio_3
dtypes: bool(1), datetime64[ns](1), float16(147), float32(3), int16(17), int32(3), int8(9)
memory usage: 101.7 MB
None
CPU times: user 38.9 s, sys: 13.8 s, total: 52.6 s
Wall time: 52.6 s


In [ ]:
train.to_pickle(train_dir+'train.pkl')
test.to_pickle(test_dir+'test.pkl')